In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import cv2
import tensorflow as tf
import os
from os.path import isfile, join

In [6]:
mypath = (r'C:\Users\ASK Computers\Desktop\CV Projects\DeepLearningCV\fruits-360\Training')
dir_names = []
dir_names = os.listdir(mypath)
print(len(dir_names))

81


In [36]:
paths = []
for dirs in dir_names:
    paths.append(join(mypath + '\\' + dirs + '\\'))
                                            


In [41]:
image_paths = []
targets = []
for target, path in enumerate(paths):
    for i in os.listdir(path):
        if isfile(join(path, i)):
            image_paths.append(join(path, i))
            targets.append(target)

In [42]:
print(len(image_paths))
print(len(targets))

41322
41322


In [67]:
train_img = []
for path in image_paths:
    img = cv2.imread(path)
    img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
    train_img.append(img)


In [75]:
train_img = np.array(train_img)
print(train_img.shape)

(41322, 100, 100, 3)


In [78]:
from keras.utils import np_utils

Using TensorFlow backend.


In [80]:
testpath = (r'C:\Users\ASK Computers\Desktop\CV Projects\DeepLearningCV\fruits-360\Test')
dir_name = []
dir_name = os.listdir(testpath)
print(len(dir_name))



81


In [84]:
path = []
for dirs in dir_name:
    path.append(join(testpath + '\\' + dirs + '\\'))
                                            


In [85]:
image_path = []
test_targets = []
for tar, pat in enumerate(path):
    for i in os.listdir(pat):
        if isfile(join(pat, i)):
            image_path.append(join(pat, i))
            test_targets.append(tar)
            
print(len(image_path))
print(len(test_targets))



13877
13877


In [86]:
test_img = []
for pa in image_path:
    img = cv2.imread(pa)
    img = cv2.resize(img, (100, 100), interpolation = cv2.INTER_AREA)
    test_img.append(img)
    
test_img = np.array(test_img)
print(test_img.shape)

(13877, 100, 100, 3)


In [90]:
shuffle_indices = np.arange(train_img.shape[0])
print(shuffle_indices.shape)

(41322,)


In [96]:
np.random.shuffle(shuffle_indices)
shuffle_indices

array([ 4931, 33788, 33328, ..., 22333, 20989, 25614])

In [97]:
shuffle_train_img = train_img[shuffle_indices]
shuffle_train_targets = np.array(targets)[shuffle_indices]

In [98]:
test_shuffle_indices = np.arange(test_img.shape[0])
print(test_shuffle_indices.shape)
np.random.shuffle(test_shuffle_indices)
shuffle_test_img = test_img[test_shuffle_indices]
shuffle_test_targets = np.array(test_targets)[test_shuffle_indices]

(13877,)


In [100]:
num_classes = 81
ytrain = np_utils.to_categorical(shuffle_train_targets, num_classes)

ytest = np_utils.to_categorical(shuffle_test_targets, num_classes)

In [102]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization

In [103]:
model = Sequential([
    
    Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (100, 100, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.20),
             
    Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.30),
    
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(81, activation = 'softmax')
    
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 32)        0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 30,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest'

)
test_datagen.fit(shuffle_test_img)
datagen.fit(shuffle_train_img)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint("/Checkpoints/fruits_fresh_cnn_1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
max_epochs = 25
batch_size = 100

history = model.fit(datagen.flow(xtrain, ytrain, batch_size = batch_size), validation_data = test_datagen.flow(xtest, ytest), steps_per_epoch = xtrain.shape[0] // batch_size, epochs = max_epochs)
